In [93]:
# Import necessary libraries


import pandas as pd
import pyodbc as db
import nltk 
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\pc\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [94]:
# Fetch data from SQL Server
# This function connects to the SQL Server database and retrieves customer reviews data
def fetch_data_from_sql():
    conn_str=(
        "Driver={ODBC Driver 17 for SQL Server};"
        "Server=DESKTOP-GJ4L9OJ\SQLEXPRESS;"
        "Database=PortfolioProject_MarketingAnalytics;"
        "Trusted_Connection=yes;"
    )
    conn = db.connect(conn_str)
    
    query = 'SELECT ReviewID, CustomerID, ProductID, ReviewDate, Rating, ReviewText FROM customer_reviews'
    df = pd.read_sql(query, conn)
    
    conn.close()
    
    return df

<>:6: SyntaxWarning: invalid escape sequence '\S'
<>:6: SyntaxWarning: invalid escape sequence '\S'
C:\Users\pc\AppData\Local\Temp\ipykernel_8680\4289361885.py:6: SyntaxWarning: invalid escape sequence '\S'
  "Server=DESKTOP-GJ4L9OJ\SQLEXPRESS;"


In [95]:

customer_reviews_df = fetch_data_from_sql()
sia = SentimentIntensityAnalyzer()

def calculate_sentiment(review_text):
    # Calculate sentiment scores using VADER
    sentiment_scores = sia.polarity_scores(review_text)
    return sentiment_scores['compound'] 

C:\Users\pc\AppData\Local\Temp\ipykernel_8680\4289361885.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [96]:
def categorize_sentiment(score, rating):
    # Categorize sentiment based on compound score
    # Positive Sentiment
    if score == 0 and rating == 3:
        return 'Neutral'
    if score >= 0 and rating >= 3:
        if rating >= 4:
            return 'Positive - High Rating'  # High Rating and Positive Sentiment
        elif rating == 3:
            return 'Positive - Neutral Rating'  # Neutral Rating and Positive Sentiment
        
    ###########################################################################################
    
    # Negative Sentiment
    elif score <= 0 and rating <= 3:
        if rating <= 2:
            return 'Negative - Low Rating'  # Low Rating and Negative Sentiment
        elif rating == 3:
            return 'Negative - Neutral Rating'  # Neutral Rating and Negative Sentiment

    ###########################################################################################
    
    # Neutral Sentiment
    else:
        return 'Neutral'  # Neutral Rating and Neutral Sentiment

In [97]:

# Apply sentiment analysis to Calculate Sentiment Score for each review
customer_reviews_df['SentimentScore'] = customer_reviews_df['ReviewText'].apply(calculate_sentiment)

# Apply sentiment categorization to each review
customer_reviews_df['SentimentCategory'] = customer_reviews_df.apply(lambda x: categorize_sentiment(x['SentimentScore'], x['Rating']), axis=1)

# Apply sentiment bucket categorization to each review by split SentimentCategory into buckets
customer_reviews_df['SentimentBucket'] =  customer_reviews_df['SentimentCategory'].apply(lambda x: x.split(' - ')[0])

In [98]:
customer_reviews_df.head(10)

,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText,SentimentScore,SentimentCategory,SentimentBucket
0,1,77,18,2023-12-23,3,"Average experience, nothing special.",-0.3089,Negative - Neutral Rating,Negative
1,2,80,19,2024-12-25,5,The quality is top-notch.,0.0000,Positive - High Rating,Positive
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.,0.0000,Positive - High Rating,Positive
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper.",0.2382,Positive - Neutral Rating,Positive
4,5,64,2,2023-07-16,3,"Average experience, nothing special.",-0.3089,Negative - Neutral Rating,Negative
5,6,81,1,2025-12-21,4,Customer support was very helpful.,0.6997,Positive - High Rating,Positive
6,7,16,1,2024-01-29,3,"Average experience, nothing special.",-0.3089,Negative - Neutral Rating,Negative
7,8,55,8,2024-08-15,5,The quality is top-notch.,0.0000,Positive - High Rating,Positive
8,9,3,13,2023-09-01,4,"I love this product, will buy again!",0.6696,Positive - High Rating,Positive
9,10,78,6,2024-06-17,5,"Excellent product, highly recommend!",0.7773,Positive - High Rating,Positive


In [99]:
# To save The Data Into CSV File
customer_reviews_df.to_csv('customer_reviews_with_sentiment.csv', index=False)